In [5]:
import pandas as pd
from datetime import datetime
import os
from pathlib import Path
import sqlite3 as sql

# Coleta de dados do site do IBGE

In [10]:
# Coletando dados do IBGE
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

dadosIBGE = pd.read_html(url, match="Municípios do Rio de Janeiro")[0]

# Renomear colunas do df
dadosIBGE = dadosIBGE.rename(columns={'Municípios do Rio de Janeiro':'nmMunic','Códigos':'codMunic'})

# Alterando o nome do index
dadosIBGE.index.name = 'idMunic'

# Alterando o index para comelar em 1
dadosIBGE.index = dadosIBGE.index + 1

# Adicionoar uma coluna de data e hora decarga 
dadosIBGE['dtCarga'] = datetime.today().strftime('%d/%m/%Y %H:%M')

# dadosIBGE

# Criando o Banco de Dados ODS

In [ ]:
# Manipulando o sistema de arquivos
path = os.getcwd()

BDODS = os.path.join(path, "pascoaODS.db")
BDDW = os.path.join(path, "pascoaDW.db")

# Manipulando os Bancos de Dados Criados

In [8]:
# Conectar no BDODS
conexaoODS = sql.connect(BDODS)

# Criar a tabela tbLogMunic e carregar os dados do DF dadosIBGE
dadosIBGE.to_sql('tbLogMunic', conexaoODS, if_exists="append")

# Confirmar a transação
conexaoODS.commit()

# Fechat a conexão
conexaoODS.close()

print("Cargo do BDODS concluída!")

92

In [ ]:
# Conectar no BDDW
conexaoDW = sql.connect(BDDW)

# Selecionar somente as colunas para criação da dMunicipio
dadosIBGE = dadosIBGE[['nmMunic', 'codMunic']]

# Criar a tabela dMunicipio e carregar os dados do DF dadosIBGE
dadosIBGE.to_sql('dMunicipio', conexaoDW, if_exists="replace")

# Confirmar a transação
conexaoDW.commit()

# Fechat a conexão
conexaoDW.close()

print("Cargo do BDDW concluída!")